# Sentan-I

Ya que el modelo propuesto por Dias es en realidad interesante, creo que vale la pena realizar una exploración de este, además aplicar los cambios propuestos desde un inicio al utilizar procesos de codificación de los datos para la reducciín de la dimensionalidad de los mismos.

Para esto se decide crear un nuevo modelo basado en el modelo de Dias, el modelo *Sentan-I*

In [1]:
%%bash
%%capture
curl https://raw.githubusercontent.com/Solrak97/clasificador_de_sentimientos/main/data.pkl > data.pkl

In [5]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

from torch.nn import Module, Conv1d, Linear, ReLU, Softmax, Maxpool1d, BatchNorm1d, Dropout, View

In [4]:
data = pd.read_pickle('data.pkl')

## Modelo base
Replica del modelo de Dias, utilizado unicamente como baseline de las pruebas.
Lamentablemente el modelo no puede estar completo al no saber la dimensionalidad de los datos de entrada por lo que su validez como replica no puede ser aceptada.

In [6]:
class Dias_Model(Module):
    
    def __init__(self):
        super(Dias_Model, self).__init__()
        
        # Capa 1
        self.conv1 = Conv1d(in_channels=1, out_channels=256, kernel_size=5, stride=1)
        self.batchnorm1 = BatchNorm1d(num_features = 7)
        self.relu1 = ReLU()
    
        # Capa 2
        self.conv2 = Conv1d(in_channels=256, out_channels=128, kernel_size=5, stride=1)
        self.relu2 = ReLU()
        self.droput1 = Dropout(p = 0.1)
        self.batchnorm2 = BatchNorm1d(num_features=42)
    
        # Maxpool
        self.maxpool = MaxPool1d(kernel_size=8)

        # Intermedias
        self.intermedia_conv1 = Conv1d(in_channels=128, out_channels=128, kernel_size=5, stride=1)
        self.intermeida_relu1 = ReLU()
    
        self.intermedia_conv2 = Conv1d(in_channels=128, out_channels=128, kernel_size=5, stride=1)
        self.intermeida_relu2 = ReLU()
    
        self.intermedia_conv3 = Conv1d(in_channels=128, out_channels=128, kernel_size=5, stride=1)
        self.intermedia_batchnorm = BatchNorm1d(num_features=7)
        self.intermeida_relu3 = ReLU()
        self.intermedia_dropout = Dropout(p=0.2)
    
        # Final
        self.final_conv = Conv1d(in_channels=128, out_channels=128, kernel_size=5, stride=1)
        self.final_dropout = Dropout(p=0.2)

        # Clasificador
        self.final_clasificador = Linear(in_features = 7, out_features=8)
        self.final_batchnorm = BatchNorm1d(num_features=8)
        self.final_softmax = Softmax()
    
        
        
        
    def forward(self, x):
        # Capa 1
        x = self.relu1(self.batchnorm1(self.conv1(x))) 
        
        # Capa 2
        x = self.batchnorm2(self.droput1(self.relu2(self.conv2(x))))
        
        # Maxpool
        x = self.maxpool(x)
        
        # Intermedias 1
        x = self.intermeida_relu1(self.intermedia_conv1(x))    
        
        # Intermedias 2
        x = self.intermeida_relu2(self.intermedia_conv2(x))
    
        # Intermedias 3
        x = self.intermedia_dropout(self.intermeida_relu3(self.intermedia_batchnorm(self.intermedia_conv3(x))))
        
        # Final
        x = self.final_dropout(self.final_conv(x).view(-1,5))
        
        # Clasificador
        x = self.final_batchnorm(self.final_clasificador(x))
        output = self.final_softmax(x)
        
        return output

## Modelo Sentan-I
Modelo creado para intentar superar al modelo de Dias basandonos en la información teorica que se brinda en su paper.

## Creación de las herramientas para evaluación del modelo

## Entrenamiento de los modelos a probar

## Evaluación de los modelos

## Recopilación de resultados

## Propuestas de mejora